In [15]:
import pandas as pd
import PIL
from glob import glob
import os
import re
import random
import tensorflow
import struct
import tensorflow as tf
import keras
import numpy as np


<font color='red'>Note:</font> Code loosely inspired by [this](https://towardsdatascience.com/board-game-image-recognition-using-neural-networks-116fc876dafa); see also [Github link](https://github.com/andrewleeunderwood/project_MYM)

<font color='red'>--> Does not work yet, only copied from the link above</font>

In [ ]:
from keras.applications.vgg16 import VGG16
model = VGG16(weights='imagenet')
model.summary()

In [20]:
from keras.preprocessing.image import ImageDataGenerator

image_size = (224, 224)
batch_size = 32
folder = 'data/final_data'

datagen = ImageDataGenerator(
        #rotation_range=5,
        rescale=1./255,
        #horizontal_flip=True,
        fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

train_gen = datagen.flow_from_directory(
    folder + '/train',
    target_size = image_size,
    batch_size = batch_size,
    class_mode = 'categorical',
    color_mode = 'rgb',
    shuffle=True)

test_gen = test_datagen.flow_from_directory(
    folder + '/test',
    target_size = image_size,
    batch_size = batch_size,
    class_mode = 'categorical',
    color_mode = 'rgb',
    shuffle=False)

Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from keras.models import Model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3)) 
 
# Freeze convolutional layers from VGG16
for layer in base_model.layers:
    layer.trainable = False
# Establish new fully connected block
x = base_model.output
x = Flatten()(x) 
x = Dense(500, activation='relu')(x) 
x = Dense(500, activation='relu')(x)
predictions = Dense(13, activation='softmax')(x)
# This is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
epochs = 10
history = model.fit(
    train_gen, 
    epochs=epochs,
    verbose = 1,
    validation_data=test_gen)